In [416]:
import time
import tqdm #tqdm은 스테이터스 바 (진행 바)
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from glob import glob
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib.request
import time
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [417]:
import warnings
warnings.filterwarnings(action = "ignore")

In [418]:
#웹 브라우저 열기
driver = webdriver.Chrome(options=chrome_options)
url = "https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tg2102.jsp?mid=S1177"
driver.get(url)

In [462]:
#지자체(구별) 크롤링 엑셀 다운로드
jijace_list_raw = driver.find_element(By.ID, "jijace") #지자체 
jijace_list = jijace_list_raw.find_elements(By.TAG_NAME, "option") #지자체 옵션
jijace_names = [option.text for option in jijace_list] #구 이름 저장
jijace_names = jijace_names[1:]

search = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div.filterForm > div.btn_wrap.btn-2 > a.btn.btn_primary")
excel_btn = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div:nth-child(2) > div.btn_box.type02 > a.btn.exbtn")

for gu in tqdm_notebook(jijace_names):
    jijace_list_raw.send_keys(gu)
    time.sleep(3)
    search.click()
    time.sleep(2)
    excel_btn.click()
    time.sleep(2)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.189)
Stacktrace:
	GetHandleVerifier [0x00007FF69D037012+3522402]
	(No symbol) [0x00007FF69CC58352]
	(No symbol) [0x00007FF69CB05ABB]
	(No symbol) [0x00007FF69CAE287C]
	(No symbol) [0x00007FF69CB75D97]
	(No symbol) [0x00007FF69CB8B3CF]
	(No symbol) [0x00007FF69CB6EE03]
	(No symbol) [0x00007FF69CB3F4D4]
	(No symbol) [0x00007FF69CB405F1]
	GetHandleVerifier [0x00007FF69D069B9D+3730157]
	GetHandleVerifier [0x00007FF69D0BF02D+4079485]
	GetHandleVerifier [0x00007FF69D0B75D3+4048163]
	GetHandleVerifier [0x00007FF69CD8A649+718233]
	(No symbol) [0x00007FF69CC64A3F]
	(No symbol) [0x00007FF69CC5FA94]
	(No symbol) [0x00007FF69CC5FBC2]
	(No symbol) [0x00007FF69CC4F2E4]
	BaseThreadInitThunk [0x00007FFA9D027344+20]
	RtlUserThreadStart [0x00007FFA9EE826B1+33]


### 서울

In [282]:
acc1_files = glob("원본데이터/서울_사고유형1/교통사고 지표 현황*xls")

tmp_raw = []
for name in acc1_files:
    tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
    tmp_raw.append(tmp)

seoul_acc1_raw = pd.concat(tmp_raw)

seoul_acc1_raw.rename(columns={'지자체': '구'}, inplace=True)

seoul_acc1_raw = seoul_acc1_raw.dropna() #결측치 삭제

seoul_acc1_raw['연도'] = seoul_acc1_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

# 피봇 테이블 직전 버전
seoul_acc1_raw.to_excel("./정제데이터/서울/서울_사고유형1_raw.xlsx") 

seoul_acc1 = seoul_acc1_raw.pivot_table( index = ["시도", "구", "연도"], columns = "사고유형1", values=['발생건수', '사망자수'], aggfunc = np.sum)
seoul_acc1

#평탄화 작업
new_columns = []

for num in range(int(len(seoul_acc1.columns)/2)):
    new_columns.append('발생_'+seoul_acc1.columns[num][1])

for num in range(int(len(seoul_acc1.columns)/2)):
    new_columns.append('사망_'+seoul_acc1.columns[num][1])

seoul_acc1.columns = new_columns

seoul_acc1.reset_index(inplace=True)

# 최종 버전
seoul_acc1.to_excel("./정제데이터/서울/서울_사고유형1.xlsx") 

In [415]:
df = pd.read_excel("./정제데이터/서울/서울_사고유형1.xlsx", index_col = 0) 
df

,시도,구,연도,발생_차대사람,발생_차대차,발생_차량단독,사망_차대사람,사망_차대차,사망_차량단독
0,서울,강남구,2018,749,2639,71,9,10,1
1,서울,강남구,2019,825,2832,65,7,6,1
2,서울,강남구,2020,721,2964,67,8,7,2
3,서울,강남구,2021,659,3103,58,6,2,2
4,서울,강남구,2022,771,2879,59,7,9,0
...,...,...,...,...,...,...,...,...,...
120,서울,중랑구,2018,432,1075,58,8,2,0
121,서울,중랑구,2019,470,1088,62,7,4,1
122,서울,중랑구,2020,358,1032,57,4,0,0
123,서울,중랑구,2021,332,1000,41,6,0,1


### 지역별 데이터 정제

In [463]:
region_list = ["서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주" ]

In [464]:
#지역 변수 선언
region = "부산"
detail = "사고유형1"

acc1_files = glob(f"원본데이터/{detail}/{region}_{detail}/교통사고 지표 현황*xls")

tmp_raw = []
for name in acc1_files:
    tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
    tmp_raw.append(tmp)

acc1_raw = pd.concat(tmp_raw)

acc1_raw.rename(columns={'지자체': '구'}, inplace=True)

acc1_raw = acc1_raw.dropna() #결측치 삭제

acc1_raw['연도'] = acc1_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

# 피봇 테이블 직전 버전
acc1_raw.to_excel(f"./정제데이터/{region}/{region}_{detail}_raw.xlsx") 

acc1 = acc1_raw.pivot_table( index = ["시도", "구", "연도"], columns = f"{detail}", values=['발생건수', '사망자수'], aggfunc = np.sum)
acc1

#평탄화 작업
new_columns = []

for num in range(int(len(acc1.columns)/2)):
    new_columns.append('발생_'+acc1.columns[num][1])

for num in range(int(len(acc1.columns)/2)):
    new_columns.append('사망_'+acc1.columns[num][1])

acc1.columns = new_columns

acc1.reset_index(inplace=True)

# 최종 버전
acc1.to_excel(f"./정제데이터/{region}/{region}_{detail}.xlsx") 

In [439]:
df = pd.read_excel(f"./정제데이터/{region}/{region}_{detail}.xlsx", index_col = 0) 
df

,시도,구,연도,발생_차대사람,발생_차대차,발생_차량단독,발생_철길건널목,사망_차대사람,사망_차대차,사망_차량단독,사망_철길건널목
0,부산,강서구,2018,61,504,19,NaN,5,8,2,NaN
1,부산,강서구,2019,69,573,27,NaN,1,6,1,NaN
2,부산,강서구,2020,55,641,33,NaN,1,5,4,NaN
3,부산,강서구,2021,57,700,26,NaN,3,10,2,NaN
4,부산,강서구,2022,59,682,24,NaN,3,8,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...
75,부산,해운대구,2018,274,717,27,NaN,7,2,3,NaN
76,부산,해운대구,2019,278,752,34,NaN,8,4,4,NaN
77,부산,해운대구,2020,196,689,29,NaN,4,4,1,NaN
78,부산,해운대구,2021,214,727,27,NaN,3,4,2,NaN


### 지역별 데이터 정제 - 업그레이드 버전

In [465]:
#지역 변수 선언
region_list = ["서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주" ]
detail = "사고유형1"

for region in region_list:
    acc1_files = glob(f"원본데이터/{detail}/{region}_{detail}/교통사고 지표 현황*xls")
    tmp_raw = []

    for name in acc1_files:
        tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
        tmp_raw.append(tmp)

    acc1_raw = pd.concat(tmp_raw)

    acc1_raw.rename(columns={'지자체': '구'}, inplace=True)

    acc1_raw = acc1_raw.dropna() #결측치 삭제

    acc1_raw['연도'] = acc1_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

    # 피봇 테이블 직전 버전
    acc1_raw.to_excel(f"./정제데이터/{region}/{region}_{detail}_raw.xlsx") 

    acc1 = acc1_raw.pivot_table( index = ["시도", "구", "연도"], columns = f"{detail}", values=['발생건수', '사망자수'], aggfunc = np.sum)
    acc1

    #평탄화 작업
    new_columns = []

    for num in range(int(len(acc1.columns)/2)):
        new_columns.append('발생_'+acc1.columns[num][1])

    for num in range(int(len(acc1.columns)/2)):
        new_columns.append('사망_'+acc1.columns[num][1])

    acc1.columns = new_columns

    acc1.reset_index(inplace=True)

    # 최종 버전
    acc1.to_excel(f"./정제데이터/{region}/{region}_{detail}.xlsx") 